In [1]:
from huggingface_hub import login

login()

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# ✅ 모델 및 토크나이저 불러오기
model_name = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# ✅ 저장할 숫자 리스트 생성
numbers = (
    [str(i) for i in range(10)] +       # "0" ~ "9"
    [f"{i:02d}" for i in range(100)] +  # "00" ~ "99"
    [f"{i:03d}" for i in range(1000)]   # "000" ~ "999"
)

# ✅ 숫자별 임베딩 벡터 저장 딕셔너리
embeddings_dict = {}

# ✅ 숫자별 임베딩 추출
for num in numbers:
    # BERT는 '[CLS] ... [SEP]' 형태의 입력을 요구함
    inputs = tokenizer(num, return_tensors="pt", add_special_tokens=True)

    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)

    # 첫 번째 토큰 ([CLS]의 히든 상태 벡터 사용)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

    # 숫자를 문자열 키로 사용하여 저장
    embeddings_dict[num] = embedding

# ✅ NumPy 파일로 저장 (NPZ 형식)
np.savez("bert_number_embeddings.npz", **embeddings_dict)

# ✅ 저장된 데이터 확인
loaded_data = np.load("bert_number_embeddings.npz")

# ✅ 결과 출력 예시 (숫자 0, 1, 99, 999의 임베딩 확인)
print("0 EMBEDDING:", loaded_data["0"])
print("1 EMBEDDING:", loaded_data["1"])
print("99 EMBEDDING:", loaded_data["99"])
print("999 EMBEDDING:", loaded_data["999"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

0 EMBEDDING: [ 0.31110913  0.06096259 -0.8033402  ...  0.22004674  0.5116869
 -0.40915757]
1 EMBEDDING: [ 0.70233744 -0.26205504 -0.53574806 ... -0.0350058   0.5037378
  0.19723356]
99 EMBEDDING: [ 0.48456523  0.38841787 -0.6901887  ... -0.04608902  1.1703607
 -0.68782574]
999 EMBEDDING: [ 0.41288865 -0.55813855 -0.29605174 ...  0.03309918  0.90096796
 -0.2308943 ]
